In [1]:
import os #for handling file and directories
from PIL import Image #image preprocesing and 
import numpy as np# store data in array
import torch#Core library for training and creating model
from torch.utils.data import Dataset, DataLoader#for loading Datasets and managing batches
import torch.nn as nn#for complex neural network architecture
import torch.optim as optim # for optimizing model's parameter
from sklearn.model_selection import train_test_split# for spliting the dataset into training and testing data
import pandas as pd
import albumentations as A
import io
from torch.nn import functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


c:\Users\shash\OneDrive\Desktop\practicing_DS\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
splits = {'train': 'Data/trainDataSet.parquet', 'test': 'Data/testDataSet.parquet'}
trainingDataSet = pd.read_parquet(splits["train"])
testingDataSet = pd.read_parquet(splits['test'])

In [3]:
T_image = testingDataSet["image"][7]['bytes']
image_stream = io.BytesIO(T_image)
image = Image.open(image_stream)
image = np.array(image)
print("Min",image.min())
print("MAX",image.max())

Min 0
MAX 255


In [4]:
""" print(type(trainingDataSet)) """
""" print(trainingDataSet.columns) """
print(testingDataSet["image"])

0       {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1       {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
2       {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3       {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
4       {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
                              ...                        
1995    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1996    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1997    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1998    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1999    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
Name: image, Length: 2000, dtype: object


In [5]:
class imageDataSet(Dataset) :
  def __init__(self,trainingDataSet,transform = None,target_transform = None):
    super().__init__()
    self.trainingDataSet_path = trainingDataSet["image"]
    self.trainingDataSet_label = trainingDataSet["labels"]
    self.transform = transform
    self.target_transform = target_transform
    
  def __len__(self):
    return len(self.trainingDataSet_path)
  
  def __getitem__(self, index):
    
    img_path = self.trainingDataSet_path.iloc[index]
    label = self.trainingDataSet_label.iloc[index]
    
    image_stream = io.BytesIO(img_path['bytes'])
    image = Image.open(image_stream).convert('RGB')
    
    if self.transform:
      image = self.transform(image)
      
    if self.target_transform:
      label = self.target_transform(label)
    
    return  image, label

In [6]:
def customMean(image_paths):
  
  pixel_sum = np.zeros(3)
  pixel_squared_sum = np.zeros(3)
  num_pixel = 0

  for img_path in image_paths:
    image = Image.open(io.BytesIO(img_path["bytes"])).convert("RGB")
    image_array = np.array(image) / np.array(image).max()  # Scale to [0, 1]
    pixel_sum += image_array.mean(axis=(0, 1))  # Sum over H, W for R, G, B
    pixel_squared_sum += (image_array ** 2).mean(axis=(0, 1))  # Squared sum
    num_pixel += 1

  mean = pixel_sum / num_pixel
  std = (pixel_squared_sum / num_pixel - mean ** 2) ** 0.5  # Standard deviation
  
  return mean.tolist(), std.tolist()

In [7]:
#Transformation (recommended)
from torchvision import transforms

image_paths = testingDataSet["image"].tolist()
mean,std = customMean(image_paths)

compose = transforms.Compose([
  transforms.Resize((64,64)),
  transforms.ToTensor(),
  transforms.Normalize(mean=mean,std=std)
])

In [8]:
# Test my imageDataSet class for errors
train_DataSet = imageDataSet(trainingDataSet=trainingDataSet,transform=compose)
image, label = next(iter(train_DataSet))
train_Loader = DataLoader(train_DataSet,batch_size = 5,shuffle=True)

test_DataSet = imageDataSet(trainingDataSet=testingDataSet,transform=compose)
image, label = next(iter(test_DataSet))
test_Loader = DataLoader(test_DataSet,batch_size=1,shuffle=True)


In [9]:
import torch.nn.functional as F

class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.residual1 = nn.Conv2d(3,32,3,4,1, bias=False)
    self.residual2 = nn.Conv2d(32,128,3,4,1,bias=False)
    
    self.conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=1,padding=1,bias=False)
    self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
    self.bn1 = nn.BatchNorm2d(16)
    
    self.conv2 = nn.Conv2d(16,32,3,1,1,bias=False)
    self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
    self.bn2 = nn.BatchNorm2d(32)
    
    
    self.conv3 = nn.Conv2d(32,64,3,1,1,bias=False)
    self.pool3 = nn.MaxPool2d(2,2,0)
    self.bn3 = nn.BatchNorm2d(64)
    
    self.conv4 = nn.Conv2d(64,128,3,1,1,bias=False)
    self.pool4 = nn.MaxPool2d(2,2,0)
    self.bn4 = nn.BatchNorm2d(128)
    
    self.flatten = nn.Flatten(start_dim=1, end_dim=-1)
    self.fc1 = nn.Linear(128*4*4,2048)
    self.dropout1 = nn.Dropout(0.4)
    self.fc2 = nn.Linear(2048,1024)
    self.dropout2 = nn.Dropout(0.2)
    self.fc3 = nn.Linear(1024,512)
    self.fc4 = nn.Linear(512,1)
    self.relu = nn.ReLU()
    
  def forward(self,x):
    res = self.residual1(x)
    
    x = self.conv1(x) 
    x = self.pool1(x)
    x = F.relu(self.bn1(x))
    
    x = self.conv2(x)
    x = self.pool2(x)
    x = F.relu(self.bn2(x))
    x = res + x
    
    res = self.residual2(x)
    x = self.conv3(x)
    x = self.pool3(x)
    x = F.relu(self.bn3(x))
    
    x = self.conv4(x)
    x = self.pool4(x)
    x = F.relu(self.bn4(x))
    x = x + res
    
    
    x = self.flatten(x)
    x = self.relu(self.fc1(x))
    x = self.dropout1(x)
    x = self.relu(self.fc2(x))
    x = self.dropout2(x)
    x = self.relu(self.fc3(x))
    x = self.fc4(x)
    
    return x
      
model = Network()   
    

In [10]:
model(torch.randn(1,3,64,64))

tensor([[-0.0557]], grad_fn=<AddmmBackward0>)

In [11]:
#Loss Function using crossEntropyLoss
loss = nn.BCEWithLogitsLoss()

In [12]:
#Optimizer using Adams
from torch.optim.lr_scheduler import ReduceLROnPlateau
optimizer = optim.AdamW(model.parameters(),lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.5, verbose=True)


In [13]:
#Evaluate Model
from tqdm import tqdm

@torch.no_grad()
def evaluate_model(test):
	model.eval()
	print("Running Validation")
	correct = 0
	for x,y in tqdm(test):
		x = x.to(device)
		y = y.view(-1, 1).float().to(device)
		out = torch.sigmoid(model(x))
		prediction = (out >= 0.5).int()
		correct += (prediction == y).sum().item()
		accuracy = (correct/len(test))*100
	return accuracy

In [14]:
#Train Model

max_accuracy = 0
EPOCHS = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for epoch in range(EPOCHS):
  sum_loss = 0
  model.train()
  for image,label in tqdm(train_Loader):
    image = image.to(device)
    label = label.to(device).float()
    
    output = model(image)
    output = output.squeeze()
    Loss = loss(output,label)
    
    optimizer.zero_grad()
    Loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    
    sum_loss += Loss.item()
    avg_loss = sum_loss/len(train_Loader)
    
  print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {sum_loss/len(train_Loader)}")
  acc = evaluate_model(test_Loader)
  print(f"Accuracy:{acc}%")
  scheduler.step(avg_loss)

100%|██████████| 1600/1600 [01:18<00:00, 20.31it/s]


Epoch [1/50], Loss: 0.606772614964284
Running Validation


100%|██████████| 2000/2000 [00:06<00:00, 294.37it/s]


Accuracy:72.39999999999999%


100%|██████████| 1600/1600 [01:18<00:00, 20.27it/s]


...

Accuracy:74.6%


100%|██████████| 1600/1600 [01:45<00:00, 15.15it/s]


Epoch [14/50], Loss: 0.09732491352789159
Running Validation


 95%|█████████▌| 1902/2000 [00:11<00:00, 158.95it/s]


KeyboardInterrupt: 

In [53]:
torch.save(model.state_dict(),"BinaryClassifier.pth")

In [54]:
image = Image.open("./dog.webp")
image_Range = np.array(image)
print("MIN",image_Range.min())
print("MAX",image_Range.max())

MIN 0
MAX 255


In [55]:
image_paths = "./dog.webp"
image = Image.open(image_paths).convert("RGB")
pixel_sum = np.zeros(3)
pixel_squared_sum = np.zeros(3)
num_pixel = 0

for img_path in image_paths:
  
  image_array = np.array(image) / 255.0  # Scale to [0, 1]
  pixel_sum += image_array.mean(axis=(0, 1))  # Sum over H, W for R, G, B
  pixel_squared_sum += (image_array ** 2).mean(axis=(0, 1))  # Squared sum
  num_pixel += 1

mean = pixel_sum / num_pixel
std = (pixel_squared_sum / num_pixel - mean ** 2) ** 0.5  # Standard deviation

print("Mean:", mean)
print("Std:", std)


Mean: [0.75128635 0.73187373 0.66569826]
Std: [0.16277556 0.16570601 0.2318707 ]
